# Example: Warp radar image

The [National Weather Service radar mosaic](http://radar.weather.gov/Conus/full.php) shows the 48 Continental states of the US. Unfortunately, it's in unprojected lat/lon coordindates. In this example, we'll use [rasterio](https://github.com/mapbox/rasterio) to reproject the image to a more standard equal-area projection.

Based on [a blog post using GDALwarp]((http://egb13.net/2009/03/using-gdalwarp) and the [rasterio reproject example](https://github.com/mapbox/rasterio/blob/master/examples/reproject.py).

In [17]:
from __future__ import print_function
import os
from IPython.display import Image
image_location = 'http://radar.weather.gov/Conus/RadarImg/latest.gif'
# uncomment the following to use local file (note: absolute paths don't work for Image())
#image_location = os.path.join(os.path.abspath('..'), 'data', 'latest.gif')

In [18]:
Image(url='http://radar.weather.gov/Conus/RadarImg/latest.gif')

In [19]:
import numpy as np
import rasterio
from rasterio.warp import reproject, RESAMPLING
from pyproj import Proj

Unfortunately, the GIF image does not have geographic metadata, so we will need to set the input projection. We get the transform parameters from the world file at http://radar.weather.gov/Conus/RadarImg/latest_radaronly.gfw

In [20]:
src_crs = {'init': 'EPSG:4326'}
west = -127.620375523875420
north = 50.406626367301044
dx = 0.017971305190311
dy = -dx

Desired width of the output image (height will be calculated):

In [21]:
width = 1600

Now we need to set the output projection. We'll use a Lambert Equal-Area projection, specifically [EPSG 2163](http://epsg.io/2163), which is used by the [National Map](http://nationalmap.gov/) to display the continental US.

Here's an example that displays average annual precipication for the CONUS:

In [22]:
Image(url='http://nationalmap.gov/small_scale/printable/images/preview/precip/pageprecip_us3.gif')

Added this bit to download the weather map locally, since the method used to open a raster over HTTP no longer works. 

In [23]:
import urllib.request
urllib.request.urlretrieve("http://radar.weather.gov/Conus/RadarImg/latest.gif", "latest.gif")

('latest.gif', <http.client.HTTPMessage at 0x7f69570855f8>)

We'll use pyproj to transform the corners of the radar image to get the bounds to use for our output grid. First we'll compute the south and east boundaries of the map in lat/lon space.  To do that, we'll open the ~~remote~~ local file with rasterio to get the image size.

Next, we'll compute the corners in projected coordinates.

In [24]:
with rasterio.drivers():
    with rasterio.open("latest.gif") as src:
        south = north + src.height * dy
        east = west + src.width * dx
        mode = 'r'
        src_transform = rasterio.transform.from_bounds(west, south, east, north, src.width, src.height)

dst_crs = {'init': 'EPSG:2163'}
us_equal_area = Proj(**dst_crs)
left, bottom = us_equal_area(west, south)
right, _ = us_equal_area(east, south)
_, top = us_equal_area(east, north)
height = width * (top - bottom) / (right - left)
dst_transform = rasterio.transform.from_bounds(left, bottom, right, top, width, height)

/home/rushj03/miniconda3/envs/jupyter/lib/python3.6/site-packages/rasterio/__init__.py:193: UserWarning: Dataset has no geotransform set.  Default transform will be applied (Affine.identity())
  s.start()


We'll initialize a NumPy array of bytes to transform the image data into.

In [25]:
dst_shape = (int(height), int(width))
destination = np.zeros(dst_shape, np.uint8)

Now for the actual transformation. We open the radar file with rasterio, and use `rasterio.warp.reproject` to transform it to our new coordinate system. It's important to use `resampling=RESAMPLING.nearest` in this case, because we don't want to interpolate values from the GIF image. For continuous data, other resampling methods may be appropriate.

In [26]:
with rasterio.drivers():
    with rasterio.open("latest.gif") as src:
        data = src.read(1)
        cmap = src.colormap(1)
        reproject(data, destination,
                  src_transform=src_transform, src_crs=src_crs,
                  dst_crs=dst_crs, dst_transform=dst_transform,
                  resampling=RESAMPLING.nearest)

/home/rushj03/miniconda3/envs/jupyter/lib/python3.6/site-packages/rasterio/__init__.py:193: UserWarning: Dataset has no geotransform set.  Default transform will be applied (Affine.identity())
  s.start()


Now the result is in a NumPy array. The values in the array are the pixel values from the GIF image, which will not make sense without that file's colormap.

We'll use rasterio to write the output to a new local GIF file.

In [27]:
with rasterio.open('warped.gif', 'w', driver='GIF',
                   width=width, height=height,
                   count=1, dtype=np.uint8) as dst:
    dst.write_band(1, destination)
    dst.write_colormap(1, cmap)

/home/rushj03/miniconda3/envs/jupyter/lib/python3.6/site-packages/rasterio/__init__.py:193: UserWarning: Dataset has no geotransform set.  Default transform will be applied (Affine.identity())
  s.start()


In [28]:
Image(url='warped.gif')

Note that we've cut off the bottom of the image because we calculated the bounding box from the corners. We could fit it in by extending the lower boundary of the output box. Alternatively, we might want to crop the image more tightly to the 48 states. Try adjusting the values of `left`, `right`, `top`, and `bottom` above that go into the output projection.

## Further exploration

- Try warping the map to a different map projection. One common example (ubiquitous on the web, though unloved by cartographers) is [EPSG 3857](https://epsg.io/3857), web Mercator.

- The National Weather Service also provides a [radar_only image](http://radar.weather.gov/Conus/RadarImg/latest_radaronly.gif) as a transparent GIF without the basemap. Try plotting your own map as a base layer and using the radar image as an overlay.